In [7]:
"""
    a test script for box-shape reconstruction
"""

import os
import sys
import shutil
import numpy as np
import torch
import utils
import vis_utils_layout as vis_utils
from data_layout import LayoutDataset, Tree
import model_layout as model
from random import shuffle

sys.setrecursionlimit(5000) # this code uses recursion a lot for code simplicity

# how many shapes to evaluate (the top-K in test.txt)
num_recon = 50

path = './logs/magazine_0.5K'
cheakpoint = ''

# load train config
conf = torch.load(path + '/conf.pth')

# load object category information
Tree.load_category_info(conf.category)
conf.device = 'cuda:1'

# set up device
device = torch.device(conf.device)
print(f'Using device: {conf.device}')

# check if eval results already exist. If so, delete it. 
out_dir = path + '/reconstructed_shapes'
if os.path.exists(out_dir):
    # response = input('result directory %s exists, overwrite? (y/n) ' % out_dir)
    # if response != 'y':
    #     sys.exit()
    shutil.rmtree(out_dir)

# create a new directory to store eval results
os.mkdir(out_dir)

# create models
# we disable probabilistic because we do not need to jitter the decoded z during inference
encoder = model.RecursiveEncoder(conf, variational=True, probabilistic=True)
decoder = model.RecursiveDecoder(conf)

# load the pretrained models
print('Loading ckpt pretrained_encoder.pth')
data_to_restore = torch.load('./' + path + '/ckpts/' + cheakpoint + 'net_encoder.pth')
encoder.load_state_dict(data_to_restore, strict=True)
print('DONE\n')
print('Loading ckpt pretrained_decoder.pth')
data_to_restore = torch.load('./' + path + '/ckpts/' + cheakpoint + 'net_decoder.pth')
decoder.load_state_dict(data_to_restore, strict=True)
print('DONE\n')

# send to device
encoder.to(device)
decoder.to(device)

# set models to evaluation mode
encoder.eval()
decoder.eval()

data_path = '/home/weiran/Projects/RvNN-Layout/data/magazine-ours/magazine_0417_0.5K/'

# read test.txt
with open(data_path + 'test.txt', 'r') as fin:
    data_list = [l.rstrip() for l in fin.readlines()]

# shuffle(data_list)

num_recon = len(data_list)

# reconstruct shapes
with torch.no_grad():
    for i in range(num_recon):
        print(f'Reconstructing {i} / {num_recon} ...')

        # load the gt data as the input
        obj = LayoutDataset.load_object(data_path + data_list[i] + '.json')
        obj.to(device)

        # feed through the encoder to get a code z
        # root_code = encoder.encode_structure(obj)
        root_code_and_kld = encoder.encode_structure(obj)
        root_code = root_code_and_kld[:, :conf.feature_size]

        # infer through the decoder to get the reconstructed output
        # set maximal tree depth to conf.max_tree_depth
        obj_arr = decoder.decode_structure(z=root_code, max_depth=conf.max_tree_depth)
        obj_arr.get_arrbox()

        # output the hierarchy
        with open(os.path.join(out_dir, data_list[i] + '_GT.txt'), 'w') as fout:
            fout.write(str(obj)+'\n\n')
            
        with open(os.path.join(out_dir, data_list[i] + '_PRED.txt'), 'w') as fout:
            fout.write(str(obj_arr)+'\n\n')

        # output the assembled box-shape
        vis_utils.draw_partnet_objects([obj], \
            object_names=['GT'], leafs_only=True, \
            sem_colors_filename='./part_colors_magazine.txt', figsize=(5, 6), \
            out_fn=os.path.join(out_dir, data_list[i] + '_GT.png'))
        
        vis_utils.draw_partnet_objects([obj], \
            object_names=['GT'], leafs_only=True, \
            sem_colors_filename='./part_colors_magazine.txt', figsize=(5, 6), \
            out_fn=os.path.join(out_dir, data_list[i] + '_GT.svg'))
        
        vis_utils.draw_partnet_objects([obj_arr], \
            object_names=['PRED'], leafs_only=True, \
            sem_colors_filename='./part_colors_magazine.txt', figsize=(5, 6), \
            out_fn=os.path.join(out_dir, data_list[i] + '_PRED.png'))

        vis_utils.draw_partnet_objects([obj_arr], \
            object_names=['PRED'], leafs_only=True, \
            sem_colors_filename='./part_colors_magazine.txt', figsize=(5, 6), \
            out_fn=os.path.join(out_dir, data_list[i] + '_PRED.svg'))

Using device: cuda:1
Loading ckpt pretrained_encoder.pth
DONE

Loading ckpt pretrained_decoder.pth
DONE

Reconstructing 0 / 55 ...
Reconstructing 1 / 55 ...
Reconstructing 2 / 55 ...
Reconstructing 3 / 55 ...
Reconstructing 4 / 55 ...
Reconstructing 5 / 55 ...
Reconstructing 6 / 55 ...
Reconstructing 7 / 55 ...
Reconstructing 8 / 55 ...
Reconstructing 9 / 55 ...
Reconstructing 10 / 55 ...
Reconstructing 11 / 55 ...
Reconstructing 12 / 55 ...
Reconstructing 13 / 55 ...
Reconstructing 14 / 55 ...
Reconstructing 15 / 55 ...
Reconstructing 16 / 55 ...
Reconstructing 17 / 55 ...
Reconstructing 18 / 55 ...
Reconstructing 19 / 55 ...
Reconstructing 20 / 55 ...
Reconstructing 21 / 55 ...
Reconstructing 22 / 55 ...
Reconstructing 23 / 55 ...
Reconstructing 24 / 55 ...
Reconstructing 25 / 55 ...
Reconstructing 26 / 55 ...
Reconstructing 27 / 55 ...
Reconstructing 28 / 55 ...
Reconstructing 29 / 55 ...
Reconstructing 30 / 55 ...
Reconstructing 31 / 55 ...
Reconstructing 32 / 55 ...
Reconstructing